Let's start with a simple example using the JsonAdapter:

In [1]:
from pydantic import BaseModel, Field
from typing import List, Optional
from pydapter.adapters.json_ import JsonAdapter


# Define a Pydantic model
class User(BaseModel):
    id: int
    name: str
    email: str
    active: bool = True
    tags: List[str] = []


# Create some test data
users = [
    User(id=1, name="Alice", email="alice@example.com", tags=["admin", "staff"]),
    User(id=2, name="Bob", email="bob@example.com", active=False),
    User(id=3, name="Charlie", email="charlie@example.com", tags=["staff"]),
]

# Convert models to JSON
json_data = JsonAdapter.to_obj(users, many=True)
print("JSON Output:")
print(json_data)

# Convert JSON back to models
loaded_users = JsonAdapter.from_obj(User, json_data, many=True)
print("\nLoaded users:")
for user in loaded_users:
    print(f"{user.name} ({user.email}): Active={user.active}, Tags={user.tags}")

JSON Output:
[
  {
    "active": true,
    "email": "alice@example.com",
    "id": 1,
    "name": "Alice",
    "tags": [
      "admin",
      "staff"
    ]
  },
  {
    "active": false,
    "email": "bob@example.com",
    "id": 2,
    "name": "Bob",
    "tags": []
  },
  {
    "active": true,
    "email": "charlie@example.com",
    "id": 3,
    "name": "Charlie",
    "tags": [
      "staff"
    ]
  }
]

Loaded users:
Alice (alice@example.com): Active=True, Tags=['admin', 'staff']
Bob (bob@example.com): Active=False, Tags=[]
Charlie (charlie@example.com): Active=True, Tags=['staff']


Using the Adaptable Mixin for Better Ergonomics

Pydapter provides an Adaptable mixin that makes the API more ergonomic:

In [2]:
from pydantic import BaseModel
from typing import List
from pydapter.core import Adaptable
from pydapter.adapters.json_ import JsonAdapter


# Define a model with the Adaptable mixin
class Product(BaseModel, Adaptable):
    id: int
    name: str
    price: float
    in_stock: bool = True


# Register the JSON adapter
Product.register_adapter(JsonAdapter)

# Create a product
product = Product(id=101, name="Laptop", price=999.99)

# Convert to JSON using the mixin method
json_data = product.adapt_to(obj_key="json")
print("JSON Output:")
print(json_data)

# Convert back to a model
loaded_product = Product.adapt_from(json_data, obj_key="json")
print(f"\nLoaded product: {loaded_product.name} (${loaded_product.price})")

JSON Output:
{
  "id": 101,
  "in_stock": true,
  "name": "Laptop",
  "price": 999.99
}

Loaded product: Laptop ($999.99)


Working with **CSV**:


In [ ]:
from pydantic import BaseModel
from pydapter.adapters.csv_ import CsvAdapter


# Define a Pydantic model
class Employee(Adaptable, BaseModel):
    id: int
    name: str
    department: str
    salary: float
    hire_date: str


# Create some sample data
employees = [
    Employee(
        id=1,
        name="Alice",
        department="Engineering",
        salary=85000,
        hire_date="2020-01-15",
    ),
    Employee(
        id=2, name="Bob", department="Marketing", salary=75000, hire_date="2021-03-20"
    ),
    Employee(
        id=3, name="Charlie", department="Finance", salary=95000, hire_date="2019-11-01"
    ),
]

csv_data = CsvAdapter.to_obj(employees, many=True)
print("CSV Output:")
print(csv_data)

# Convert CSV back to models
loaded_employees = CsvAdapter.from_obj(Employee, csv_data, many=True)
print("\nLoaded employees:")
for employee in loaded_employees:
    print(f"{employee.name} - {employee.department} (${employee.salary})")

# You can also save to a file and read from a file
from pathlib import Path

# Save to file
Path("employees.csv").write_text(csv_data)

# Read from file
file_employees = CsvAdapter.from_obj(Employee, Path("employees.csv"), many=True)

CSV Output:
id,name,department,salary,hire_date
1,Alice,Engineering,85000.0,2020-01-15
2,Bob,Marketing,75000.0,2021-03-20
3,Charlie,Finance,95000.0,2019-11-01


Loaded employees:
Alice - Engineering ($85000.0)
Bob - Marketing ($75000.0)
Charlie - Finance ($95000.0)


In [13]:
file_employees;

[Employee(id=1, name='Alice', department='Engineering', salary=85000.0, hire_date='2020-01-15'),
 Employee(id=2, name='Bob', department='Marketing', salary=75000.0, hire_date='2021-03-20'),
 Employee(id=3, name='Charlie', department='Finance', salary=95000.0, hire_date='2019-11-01')]

Working with **TOML**

In [4]:
from pydantic import BaseModel
from typing import List, Dict, Optional
from pydapter.adapters.toml_ import TomlAdapter


# Define a Pydantic model
class AppConfig(BaseModel):
    app_name: str
    version: str
    debug: bool = False
    database: Dict[str, str] = {}
    allowed_hosts: List[str] = []


# Create a config
config = AppConfig(
    app_name="MyApp",
    version="1.0.0",
    debug=True,
    database={"host": "localhost", "port": "5432", "name": "myapp"},
    allowed_hosts=["localhost", "example.com"],
)

# Convert to TOML
toml_data = TomlAdapter.to_obj(config)
print("TOML Output:")
print(toml_data)

# Convert TOML back to model
loaded_config = TomlAdapter.from_obj(AppConfig, toml_data)
print("\nLoaded config:")
print(f"App: {loaded_config.app_name} v{loaded_config.version}")
print(f"Debug mode: {loaded_config.debug}")
print(f"Database: {loaded_config.database}")
print(f"Allowed hosts: {loaded_config.allowed_hosts}")

# Save to file
Path("config.toml").write_text(toml_data)

# Read from file
file_config = TomlAdapter.from_obj(AppConfig, Path("config.toml"))

TOML Output:
app_name = "MyApp"
version = "1.0.0"
debug = true
allowed_hosts = [ "localhost", "example.com",]

[database]
host = "localhost"
port = "5432"
name = "myapp"


Loaded config:
App: MyApp v1.0.0
Debug mode: True
Database: {'host': 'localhost', 'port': '5432', 'name': 'myapp'}
Allowed hosts: ['localhost', 'example.com']


Working with **Pandas DataFrame**

`pip install "pydapter[pandas]"` to use the pandas adapter.

In [5]:
import pandas as pd
from pydantic import BaseModel
from pydapter.extras.pandas_ import DataFrameAdapter


# Define a Pydantic model
class SalesRecord(BaseModel):
    id: int
    product: str
    quantity: int
    price: float
    date: str


# Create a sample DataFrame
df = pd.DataFrame(
    [
        {
            "id": 1,
            "product": "Laptop",
            "quantity": 2,
            "price": 999.99,
            "date": "2023-01-15",
        },
        {
            "id": 2,
            "product": "Monitor",
            "quantity": 3,
            "price": 249.99,
            "date": "2023-01-20",
        },
        {
            "id": 3,
            "product": "Mouse",
            "quantity": 5,
            "price": 29.99,
            "date": "2023-01-25",
        },
    ]
)

# Convert DataFrame to models
sales_records = DataFrameAdapter.from_obj(SalesRecord, df, many=True)
print("DataFrame to Models:")
for record in sales_records:
    print(f"{record.id}: {record.quantity} x {record.product} at ${record.price}")

# Convert models back to DataFrame
new_df = DataFrameAdapter.to_obj(sales_records, many=True)
print("\nModels to DataFrame:")
print(new_df)

DataFrame to Models:
1: 2 x Laptop at $999.99
2: 3 x Monitor at $249.99
3: 5 x Mouse at $29.99

Models to DataFrame:
   id  product  quantity   price        date
0   1   Laptop         2  999.99  2023-01-15
1   2  Monitor         3  249.99  2023-01-20
2   3    Mouse         5   29.99  2023-01-25


Working with **Excel** Files

`pip install "pydapter[excel]"` to use the excel adapter.

In [6]:
from pydantic import BaseModel
from typing import List, Optional
from pydapter.extras.excel_ import ExcelAdapter
from pathlib import Path


# Define a Pydantic model
class Student(BaseModel):
    id: int
    name: str
    grade: str
    score: float


# Create some sample data
students = [
    Student(id=1, name="Alice", grade="A", score=92.5),
    Student(id=2, name="Bob", grade="B", score=85.0),
    Student(id=3, name="Charlie", grade="A-", score=90.0),
]

# Convert to Excel and save to file
excel_data = ExcelAdapter.to_obj(students, many=True, sheet_name="Students")
with open("students.xlsx", "wb") as f:
    f.write(excel_data)

print("Excel file saved as 'students.xlsx'")

# Read from Excel file
loaded_students = ExcelAdapter.from_obj(Student, Path("students.xlsx"), many=True)
print("\nLoaded students:")
for student in loaded_students:
    print(f"{student.name}: {student.grade} ({student.score})")

Excel file saved as 'students.xlsx'

Loaded students:
Alice: A (92.5)
Bob: B (85.0)
Charlie: A- (90.0)


**Error handling**

In [8]:
from pydantic import BaseModel, Field
from pydapter.adapters.json_ import JsonAdapter
from pydapter.exceptions import ParseError, ValidationError as AdapterValidationError


# Define a model with validation constraints
class Product(BaseModel):
    id: int = Field(gt=0)  # Must be greater than 0
    name: str = Field(min_length=3)  # Must be at least 3 characters
    price: float = Field(gt=0.0)  # Must be greater than 0


# Handle parsing errors
try:
    # Try to parse invalid JSON
    invalid_json = "{ 'id': 1, 'name': 'Laptop', price: 999.99 }"  # Note the missing quotes around 'price'
    product = JsonAdapter.from_obj(Product, invalid_json)
except ParseError as e:
    print(f"Parsing error: {e}")

# Handle validation errors
try:
    # Try to create a model with invalid data
    valid_json = (
        '{"id": 0, "name": "A", "price": -10.0}'  # All fields violate constraints
    )
    product = JsonAdapter.from_obj(Product, valid_json)
except AdapterValidationError as e:
    print(f"Validation error: {e}")
    if hasattr(e, "errors") and callable(e.errors):
        for error in e.errors():
            print(f"  - {error['loc']}: {error['msg']}")

Parsing error: Invalid JSON: Expecting property name enclosed in double quotes: line 1 column 3 (char 2) (position=2, line=1, column=3)
Validation error: Validation error: 3 validation errors for Product
id
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than
name
  String should have at least 3 characters [type=string_too_short, input_value='A', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
price
  Input should be greater than 0 [type=greater_than, input_value=-10.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than (errors=[{'type': 'greater_than', 'loc': ('id',), 'msg': 'Input should be greater than 0', 'input': 0, 'ctx': {'gt': 0}, 'url': 'https://errors.pydantic.dev/2.11/v/greater_than'}, {'type': 'string_too_short', 'loc': ('name',), 'msg': 'String should have at l